<a href="https://colab.research.google.com/github/ik-okoro/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module3-Tune/LS_DS_423_Tune_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [1]:
from google.colab import files
uploaded = files.upload()

Saving quickdraw10.npz to quickdraw10.npz


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
def load_quickdraw10(path = "quickdraw10.npz"):
  """
  Takes a path for the quickdraw data sample and standardizes
  Returns: X_train, X_test, Y-train, y_test
  """
  data = np.load(path)
  X = data["arr_0"].astype("float32")/255
  y = data["arr_1"]

  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)  

  return X_train, y_train, X_test, y_test

In [4]:
X_train, y_train, X_test, y_test = load_quickdraw10()

In [5]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.15686275, 0.29411766, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.02352941, 0.83137256, 0.8039216 , 0.42352942, 0.3882353 ,
       0.8352941 , 0.93333334, 0.6431373 , 0.26666668, 0.     

In [6]:
print(X_train.shape)
print(y_train.shape)

(75000, 784)
(75000,)


In [7]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adadelta, Adagrad, Adam, RMSprop, SGD
!pip install keras-tuner
from kerastuner import HyperModel, HyperParameters
from kerastuner.tuners import RandomSearch

     |████████████████████████████████| 71kB 6.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=ba83401c2737e396247b83fcf8ec2372e083a71779b683f84e8247e350a6d311
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=35f7bcc107ecf76fab25f766dfeee64cb9879065db6084f323c302bd6fab50d7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [25]:
from tensorflow.keras.activations import relu, selu
from tensorflow.keras.initializers import Constant, zeros, ones

In [36]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Lambda

In [11]:
const_init = Constant(3)

In [44]:
class NNModel(HyperModel):
  """
  Classification hypertuning that inherits HyperModel
  Pass in number of classes. Default is 10 in this case
  """

  def __init__(self, num_classes = 10):
    self.num_classes = num_classes
  
  def build(self, hp):

    # Set class variables for parameter ranges
    #optimizer = hp.Choice("optimizer", values = [""])
    learning_rate = hp.Choice("learning_rate", values = [0.001, 0.005, 0.0005, 0.0025, 0.00075], default = 0.001)
    activation = hp.Choice("activation", values = ["relu", "swish", "selu"], default = "relu")
    relu_alpha = hp.Float("relu_alpha", 0.0, 0.5, 0.1, default = 0.0)
    layers = hp.Int("hiddden_layers", 1, 5, default = 2)
    kernel_init = hp.Choice("kernel_init", values = ["random_normal", "random_uniform", "truncated_normal", "glorot_uniform", "glorot_normal"], default = "glorot_normal")
    bias_init = hp.Choice("bias_init", values = ["zeros", "ones"], default = "zeros")

    model = Sequential()

    model.add(Dense(hp.Int("units", 16, 96, 16), activation = Activation(Lambda((lambda x: relu(x, alpha= relu_alpha)))), input_dim = 784, kernel_initializer= kernel_init, bias_initializer= bias_init,))

    for i in range(layers):
      model.add(Dense(hp.Int("units_" + str(i), 16, 96, 16), activation= activation, kernel_initializer= kernel_init, bias_initializer= bias_init))
    
    model.add(Dense(self.num_classes, activation= "softmax"))

    model.compile(optimizer = Adam(learning_rate), loss = "sparse_categorical_crossentropy", metrics = "accuracy")

    return model

In [45]:
nn_model = NNModel()

In [46]:
tuner = RandomSearch(
    nn_model,
    objective = "val_accuracy",
    max_trials = 15,
    seed = 42,
    executions_per_trial = 3,
    directory = "./quickdraw-tuner",
    project_name = "quickdraw"
)

INFO:tensorflow:Reloading Oracle from existing project ./quickdraw-tuner/quickdraw/oracle.json


In [47]:
tuner.search(X_train, y_train, epochs = 50, validation_data = (X_test, y_test))

Trial 15 Complete [00h 09m 41s]
val_accuracy: 0.8590533336003622

Best val_accuracy So Far: 0.863319993019104
Total elapsed time: 02h 26m 07s
INFO:tensorflow:Oracle triggered exit


In [48]:
tuner.results_summary()

Results summary
Results in ./quickdraw-tuner/quickdraw
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.001
activation: swish
relu_alpha: 0.4
hiddden_layers: 4
kernel_init: truncated_normal
bias_init: ones
units: 80
units_0: 48
units_1: 16
units_2: 96
units_3: 96
units_4: 96
Score: 0.863319993019104
Trial summary
Hyperparameters:
learning_rate: 0.0025
activation: swish
relu_alpha: 0.1
hiddden_layers: 3
kernel_init: glorot_uniform
bias_init: zeros
units: 80
units_0: 80
units_1: 96
units_2: 96
units_3: 32
units_4: 64
Score: 0.8632400035858154
Trial summary
Hyperparameters:
learning_rate: 0.0005
activation: relu
relu_alpha: 0.2
hiddden_layers: 3
kernel_init: random_uniform
bias_init: zeros
units: 32
units_0: 96
units_1: 96
units_2: 80
units_3: 80
units_4: 32
Score: 0.8620933294296265
Trial summary
Hyperparameters:
learning_rate: 0.001
activation: relu
relu_alpha: 0.2
hiddden_layers: 2
kernel_init: random_normal
bias_ini

In [49]:
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(X_test, y_test)

782/782 [==============================] - 1s 918us/step - loss: 0.4610 - accuracy: 0.8636


### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 